# **3. Generación de Lenguaje Natural**

El objetivo es el de simular la creación de un bot que escriba tweets de manera similar a **Barack Obama**

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [ ]:
import numpy as np
import pandas as pd
from copy import deepcopy

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

Using TensorFlow backend.


## **Carga de datos**

In [ ]:
# montamos GDrive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/datos"

BarackObama.csv  model_1.h5  model_2.h5  model_3.h5  model_4.h5


In [ ]:
import pandas as pd

In [ ]:

copied_path = '/content/drive/My Drive/datos/BarackObama.csv'
data = pd.read_csv(copied_path)

In [ ]:
data.head()

,Unnamed: 0,date,id,link,retweet,text,author
0,0,20h20 hours ago,786982739517943808,/BarackObama/status/786982739517943808,False,Denying climate change is dangerous. Join @OFA...,BarackObama
1,1,18h18 hours ago,787010142378332160,/BarackObama/status/787010142378332160,False,The American Bar Association gave Judge Garlan...,BarackObama
2,2,16h16 hours ago,787039774330748928,/BarackObama/status/787039774330748928,False,We need a fully functional Supreme Court. Edit...,BarackObama
3,3,21h21 hours ago,786964419905523712,/BarackObama/status/786964419905523712,False,"Cynics, take note: When we #ActOnClimate, we b...",BarackObama
4,4,Oct 13,786680553617629184,/BarackObama/status/786680553617629185,False,"""That’s how we will overcome the challenges we...",BarackObama


In [ ]:
data.shape

(6896, 7)

Se han cargado 6896 tweets y nos vamos a uqedar con la columna **text**

In [ ]:
data = data[['text']]
data.dropna(inplace=True)

In [ ]:
data.head()

,text
0,Denying climate change is dangerous. Join @OFA...
1,The American Bar Association gave Judge Garlan...
2,We need a fully functional Supreme Court. Edit...
3,"Cynics, take note: When we #ActOnClimate, we b..."
4,"""That’s how we will overcome the challenges we..."


## **Limpieza de los tweets**

Aquí pasamos todo es texto a minúsculas, limpiamos los tweets de hastag, menciones, urls y después limpiamos el texto de caracteres extraños o que creo yo que en un documento no deberían de ir y se crea una nueva columna llamada **text_clean**

In [ ]:
regex = "\s([\w_-]+).pic.twitter.com\\S+"

In [ ]:
regex1 = "\s.twitter.com\\S+"

In [ ]:
regex2 = "\s.pic.twitter.com\\S+"

In [ ]:
regex3 = "pic.twitter.com\\S+"

In [ ]:
regex4="\[([^]]+)\]"

In [ ]:
regex5="[^a-z ,.?!’]+"

In [ ]:
import re
def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\s([@#＃][\w_-]+)", " ", elem))  
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"https?:\/\/\S*\xa0", "", elem))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(regex, "", elem))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(regex1, "", elem))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(regex2, "", elem))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(regex3, "", elem))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(regex4, "", elem))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(regex5, "", elem))
    
    
    return df
data_clean = clean_text(data, 'text', 'text_clean')
data_clean.head()

,text,text_clean
0,Denying climate change is dangerous. Join @OFA...,denying climate change is dangerous. join sup...
1,The American Bar Association gave Judge Garlan...,the american bar association gave judge garlan...
2,We need a fully functional Supreme Court. Edit...,we need a fully functional supreme court. edit...
3,"Cynics, take note: When we #ActOnClimate, we b...","cynics, take note when we , we build a stronger"
4,"""That’s how we will overcome the challenges we...",that’s how we will overcome the challenges we ...


In [ ]:
text = '\n'.join(list(data['text_clean'].values))

Ahora unimos todos los tweets en un corpus separandolos por un retorno de carro. Vemos que la longitud de nuestro corpus es 634082

In [ ]:
print('corpus length: {}'.format(len(text)))

corpus length: 634082


In [ ]:
text[:10000]

'denying climate change is dangerous. join  supporters in standing up for bold action now  \nthe american bar association gave judge garland its highest rating. senate leaders  and fill the \nwe need a fully functional supreme court. editorial boards across the country agree  \ncynics, take note when we , we build a stronger\nthat’s how we will overcome the challenges we face by unleashing the power of all of us for all of us. read more\ncheck out the most recent editorial boards calling on senate leaders to vote on judge garland  \nsenate leaders political games are handicapping the supreme courtand judgeships across the country.  \nsupport the bold steps were taking against climate change. say youre ready to  \nlast week, more than  countries reached a landmark deal to reduce carbon pollution from aviation.  \nlets leave our children and grandchildren a planet thats healthier than the one we have today.\nclimate changes effects are devastating. denying the science behind it does noth

Para realizar el entrenamiento voy areducir el corpus a 100000 caracteres ya que con el corpus entero tardaba mucho tiempo en el entrenamiento.

In [ ]:
text=text[:100000]

In [ ]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

print('Unique chars: {}'.format(len(characters)))

Unique chars: 33


In [ ]:
print(characters)

['\n', ' ', '!', ',', '.', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '’']


In [ ]:
print(n_to_char)

{0: '\n', 1: ' ', 2: '!', 3: ',', 4: '.', 5: '?', 6: 'a', 7: 'b', 8: 'c', 9: 'd', 10: 'e', 11: 'f', 12: 'g', 13: 'h', 14: 'i', 15: 'j', 16: 'k', 17: 'l', 18: 'm', 19: 'n', 20: 'o', 21: 'p', 22: 'q', 23: 'r', 24: 's', 25: 't', 26: 'u', 27: 'v', 28: 'w', 29: 'x', 30: 'y', 31: 'z', 32: '’'}


## **Preprocesado de los datos**

In [ ]:
X = []
Y = []
length = len(text)
seq_length = 30
for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [ ]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

## **Modelos**

**Modelo 1**

In [ ]:
model_1 = Sequential()
model_1.add(LSTM(512, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model_1.add(Dropout(0.2))
model_1.add(LSTM(512))
model_1.add(Dropout(0.2))
model_1.add(Dense(Y_modified.shape[1], activation='softmax'))
model_1.compile(loss='categorical_crossentropy', optimizer='adam')
history_1 = model_1.fit(X_modified, Y_modified, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history
model_1.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 94971 samples, validate on 4999 samples
Epoch 1/20
94971/94971 [==============================] - 104s 1ms/step - loss: 2.8592 - val_loss: 2.5759
Epoch 2/20
94971/94971 [==============================] - 102s 1ms/step - loss: 2.4843 - val_loss: 2.2575
Epoch 3/20
94971/94971 [==============================] - 102s 1ms/step - loss: 2.1898 - val_loss: 1.9834
Epoch 4/20
94971/94971 [==============================] - 102s 1ms/step - loss: 1.9350 - val_loss: 1.7885
Epoch 5/20
94971/94971 [==============================] - 102s 1ms/step - loss: 1.7468 - val_loss: 1.6907
Epoch 6/20
94971/94971 [==============================] - 101s 1ms/step - loss: 1.5977 - val_loss: 1.6442
Epoch 7/20
94971/94971 [==============================] - 101s 1ms/step - loss: 1.4822 - val_loss: 1.5759
Epoch 8/20
94971/94971 [==============================] - 101s 1ms/step - loss: 1.3769 - val_loss: 1.5701
Epoch 9/20
94971/9497

**Modelo 2**

Varia del primero en las épocas de entrenamiento, en lugar de 20 pasamos 40.

In [ ]:
# Model 2
model_2 = Sequential()
model_2.add(LSTM(512, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model_2.add(Dropout(0.2))
model_2.add(LSTM(512))
model_2.add(Dropout(0.2))
model_2.add(Dense(Y_modified.shape[1], activation='softmax'))
model_2.compile(loss='categorical_crossentropy', optimizer='adam')
history_2 = model_2.fit(X_modified, Y_modified, validation_split=0.05, batch_size=128, epochs=40, shuffle=True).history
model_2.summary()

Train on 94971 samples, validate on 4999 samples
Epoch 1/40
94971/94971 [==============================] - 102s 1ms/step - loss: 2.8594 - val_loss: 2.5978
Epoch 2/40
94971/94971 [==============================] - 101s 1ms/step - loss: 2.4943 - val_loss: 2.2781
Epoch 3/40
94971/94971 [==============================] - 101s 1ms/step - loss: 2.2044 - val_loss: 1.9996
Epoch 4/40
94971/94971 [==============================] - 101s 1ms/step - loss: 1.9519 - val_loss: 1.8297
Epoch 5/40
94971/94971 [==============================] - 101s 1ms/step - loss: 1.7583 - val_loss: 1.7020
Epoch 6/40
94971/94971 [==============================] - 101s 1ms/step - loss: 1.6095 - val_loss: 1.6549
Epoch 7/40
94971/94971 [==============================] - 101s 1ms/step - loss: 1.4902 - val_loss: 1.6185
Epoch 8/40
94971/94971 [==============================] - 101s 1ms/step - loss: 1.3771 - val_loss: 1.6020
Epoch 9/40
94971/94971 [==============================] - 101s 1ms/step - loss: 1.2825 - val_loss: 1.58

**Modelo 3**

In [ ]:
model_3 = Sequential()
model_3.add(LSTM(512, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model_3.add(Dropout(0.2))
model_3.add(LSTM(512, return_sequences=True))
model_3.add(Dropout(0.2))
model_3.add(LSTM(512))
model_3.add(Dropout(0.2))
model_3.add(Dense(Y_modified.shape[1], activation='softmax'))
model_3.compile(loss='categorical_crossentropy', optimizer='adam')
history_3 = model_3.fit(X_modified, Y_modified, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history
model_3.summary()

Train on 94971 samples, validate on 4999 samples
Epoch 1/20
94971/94971 [==============================] - 160s 2ms/step - loss: 2.8870 - val_loss: 2.5500
Epoch 2/20
94971/94971 [==============================] - 158s 2ms/step - loss: 2.4080 - val_loss: 2.1261
Epoch 3/20
94971/94971 [==============================] - 158s 2ms/step - loss: 2.0529 - val_loss: 1.7997
Epoch 4/20
94971/94971 [==============================] - 158s 2ms/step - loss: 1.7943 - val_loss: 1.6755
Epoch 5/20
94971/94971 [==============================] - 158s 2ms/step - loss: 1.6117 - val_loss: 1.5641
Epoch 6/20
94971/94971 [==============================] - 158s 2ms/step - loss: 1.4753 - val_loss: 1.5002
Epoch 7/20
94971/94971 [==============================] - 158s 2ms/step - loss: 1.3567 - val_loss: 1.4811
Epoch 8/20
94971/94971 [==============================] - 158s 2ms/step - loss: 1.2523 - val_loss: 1.4647
Epoch 9/20
94971/94971 [==============================] - 158s 2ms/step - loss: 1.1533 - val_loss: 1.44

## **Generación de texto**

Vamos a generar tweets de 200 caracteres

In [ ]:
def generate_text(model, string_id):
    string_mapped = deepcopy(string_id)
    full_string = [n_to_char[value] for value in string_mapped]
    
    # Generating characters
    for i in range(200):
        x = np.reshape(string_mapped,(1,len(string_mapped), 1))
        x = x / float(len(characters))

        pred_index = np.argmax(model.predict(x, verbose=0))
        seq = [n_to_char[value] for value in string_mapped]
        full_string.append(n_to_char[pred_index])

        string_mapped.append(pred_index)
        string_mapped = string_mapped[1:len(string_mapped)]
        
    text = ""
    for char in full_string:
        text = text + char
    return text

Generamos el texto cn cada uno de los modelos definidos y entrenados

In [ ]:
model_1_results = generate_text(model_1, X[10])

In [ ]:
model_2_results = generate_text(model_2, X[10])

In [ ]:
model_3_results = generate_text(model_3, X[10])

## **Presentación de resultados**

Nuestra frase entrada es X[10]

In [ ]:
t = ''
for ch in X[10]:
    t += n_to_char.get(ch)
print(t)

imate change is dangerous. joi


In [ ]:
print(model_1_results)

imate change is dangerous. join  senate leaders oeed to do their jobs   sign the conseruation to be the hintre weer ee the sane with dongress to  about gun violence in the w.y. senperates jobs the senate leaders to give judge garl


En este primer modelo se observa que escribe texto, pero que la mayoría de las palabras son inventadas. 

In [ ]:
print(model_2_results)

imate change is dangerous. join  supporters in standing on toorecedented. sell she senate .
with more than  million americans have gained coverages contri, rrasion our mat on the soiar industry hit teeord growth last year, surpats


En este segundo modelo con más épocas de entrenamiento, vemos que aunque hay todavía palabras inventadas, podemos ver como mete en el texto punto y aparte y comas.

In [ ]:
print(model_3_results)

imate change is dangerous. join  supporting americans without the security of health coverage. 
in the weekly address, the president just remected . she best scientists in the world in this congress a bold new plan to lake sure th


Vemos que con este último modelo, se observa que mayoria de las palabras son comprensibles y podemos que las frases tiene algo de coherencia.

Podríamos decir como conclusión que quizas con un modelo de mas capas y mas épocas, y uilizando en corpus mas grande quizá se mejoraría el texto.